In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "../00_setup.py").load_module()

In [2]:
setup.temp_path

'../data/2023_12_15'

# Neural Network Classification Test - Basic

The purpose of this script is to build a standard (not graph) NN and try to get decent performance.  I want to test the data prepared in script 10

Some code based on: https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

https://github.com/stellargraph/stellargraph/issues/1908

*This script takes about 1 hour on my MacBook Air*

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
from pathlib import Path
import importlib, pickle
from sklearn import model_selection

In [5]:
import pandas as pd
import os

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

2024-01-05 20:47:37.096001: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-05 20:47:37.096032: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-05 20:47:37.096038: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-05 20:47:37.096086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-05 20:47:37.096106: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.metrics as km

In [7]:
import tensorflow.keras.metrics as km
from keras import initializers

## Input Data

In [8]:
os.chdir('..')

In [9]:
os.getcwd()

'/Users/valeriecarey/Documents/projects/2023_10_blog_gnn_sba/code'

In [10]:
business_data = pd.read_parquet(Path(setup.temp_path).joinpath('TMP_DATA_combined_scaled_all.parquet'))

In [11]:
business_data.describe()

,target,menc_NAICS,menc_NAICS_sector,NS___Accommodation and Food Services,NS___Construction,NS___Health Care and Social Assistance,NS___Manufacturing,NS___Other Services (except Public Administration),"NS___Professional, Scientific, and Technical Services",NS___Retail Trade,...,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NoEmp,CreateJob,LowDoc,DisbursementGross
count,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,...,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,688081.000000,6.880810e+05,6.880810e+05,6.880810e+05,6.880810e+05
mean,0.203011,0.203954,0.202926,0.097958,0.095606,0.079398,0.096843,0.103658,0.098048,0.182085,...,0.277935,0.852639,0.050324,0.007342,0.001235,0.198575,6.691533e-18,-7.517649e-18,-2.368885e-17,-5.093000e-17
std,0.402241,0.095249,0.043782,0.297258,0.294051,0.270359,0.295744,0.304816,0.297380,0.385915,...,0.447982,0.354465,0.218613,0.085371,0.035125,0.398928,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,0.000000,0.009628,0.068505,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.824568e-01,-2.086068e-01,-3.314259e-01,-6.420047e-01
25%,0.000000,0.132721,0.192547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-1.364398e-01,-1.674320e-01,-3.291739e-01,-5.312204e-01
50%,0.000000,0.200864,0.218455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-1.051958e-01,-1.323106e-01,-3.287489e-01,-3.501667e-01
75%,0.000000,0.264205,0.228565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-1.235818e-02,-1.341020e-02,-3.287489e-01,7.064531e-02
max,1.000000,0.488372,0.286731,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.560525e+02,3.423551e+02,3.041835e+00,3.773855e+01


In [12]:
business_data.columns

Index(['dset', 'LoanNr_ChkDgt', 'target', 'NAICS', 'NAICS_sector',
       'menc_NAICS', 'menc_NAICS_sector',
       'NS___Accommodation and Food Services', 'NS___Construction',
       'NS___Health Care and Social Assistance', 'NS___Manufacturing',
       'NS___Other Services (except Public Administration)',
       'NS___Professional, Scientific, and Technical Services',
       'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn',
       'new_business', 'urban_flag', 'franchise_flag',
       'missingindicator_LowDoc', 'missingindicator_new_business',
       'missingindicator_urban_flag', 'NoEmp', 'CreateJob', 'LowDoc',
       'DisbursementGross'],
      dtype='object')

In [13]:
feat = ['new_business','urban_flag', 'franchise_flag','missingindicator_LowDoc',
        'missingindicator_new_business', 'missingindicator_urban_flag', 'NoEmp',
        'CreateJob', 'LowDoc', 'DisbursementGross']


In [14]:
X = business_data[['dset', 'LoanNr_ChkDgt'] + feat].set_index('LoanNr_ChkDgt').sort_index()

In [15]:
X.shape

(688081, 11)

In [16]:
X.columns

Index(['dset', 'new_business', 'urban_flag', 'franchise_flag',
       'missingindicator_LowDoc', 'missingindicator_new_business',
       'missingindicator_urban_flag', 'NoEmp', 'CreateJob', 'LowDoc',
       'DisbursementGross'],
      dtype='object')

## Function to create model

In [17]:
def create_baseline(base_thresh  = 0.2, n_feat = 10, hidden_size = 32):
    model = Sequential()
    #model.add(Dense(60, input_shape=(7,), activation='relu'))
    #model.add(Dense(2, input_shape=(10,), activation='relu'))
    model.add(Dense(hidden_size, input_shape=(10,), activation='sigmoid'))
    #model.add(Dense(hidden_size, input_shape=(hidden_size,), activation='selu'))
    model.add(Dense(hidden_size, input_shape=(hidden_size,), activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                  metrics=[km.AUC(curve='PR'), km.AUC(curve='ROC'), km.Precision(thresholds=base_thresh), 
                           km.Recall(thresholds=base_thresh)])
    return model

## Model Create, Fit

In [18]:
X_train = X[X['dset'] == 'train'].drop(columns='dset')
X_train.shape

(412848, 10)

In [19]:
X_val = X[X['dset'] == 'val'].drop(columns='dset')
X_val.shape

(220187, 10)

In [20]:
y_train = business_data[business_data['dset'] == 'train'].set_index('LoanNr_ChkDgt').sort_index()['target']

In [21]:
y_train.shape

(412848,)

In [22]:
y_val = business_data[business_data['dset'] == 'val'].set_index('LoanNr_ChkDgt').sort_index()['target']

In [23]:
model = create_baseline()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=15
)

Epoch 1/15


2024-01-05 20:47:40.253760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


12902/12902 [==============================] - ETA: 0s - loss: 0.4708 - auc: 0.3028 - auc_1: 0.6762 - precision: 0.2860 - recall: 0.7787

2024-01-05 20:51:49.346267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


12902/12902 [==============================] - 326s 25ms/step - loss: 0.4708 - auc: 0.3028 - auc_1: 0.6762 - precision: 0.2860 - recall: 0.7787 - val_loss: 0.4676 - val_auc: 0.3099 - val_auc_1: 0.6834 - val_precision: 0.2889 - val_recall: 0.7829
Epoch 2/15
12902/12902 [==============================] - 304s 24ms/step - loss: 0.4680 - auc: 0.3088 - auc_1: 0.6820 - precision: 0.2916 - recall: 0.7642 - val_loss: 0.4667 - val_auc: 0.3124 - val_auc_1: 0.6848 - val_precision: 0.2947 - val_recall: 0.7505
Epoch 3/15
12902/12902 [==============================] - 305s 24ms/step - loss: 0.4670 - auc: 0.3117 - auc_1: 0.6843 - precision: 0.2923 - recall: 0.7613 - val_loss: 0.4660 - val_auc: 0.3158 - val_auc_1: 0.6865 - val_precision: 0.2925 - val_recall: 0.7642
Epoch 4/15
12902/12902 [==============================] - 302s 23ms/step - loss: 0.4664 - auc: 0.3137 - auc_1: 0.6855 - precision: 0.2921 - recall: 0.7643 - val_loss: 0.4655 - val_auc: 0.3179 - val_auc_1: 0.6871 - val_precision: 0.2956 - va

  * 60 deep layer - at most 0.27 / 0.28 after 2-3 epochs
  * went down to.2 neurons in single hidden layer. Added the missing indicators. Get better performance. Seems to train a bit. get to 0.306/0.306 after 3 epochs
  * Try linear activation in intermediate layers.  Worse at first.  May be better at epoch 2?  0.307/0.304 after 3 epochs
  * Added a layer, selu then linear.  2 neurons each.  This is better slightly.  Let it run a bit.  3 levels get to 0.311/0.310
  * Same bt make hidden size 4 for both layers.  Now 0.36/0.33 after 10 epochs.  Seems to train
  * Add another selu lauyer (selu, selu, linear, sigmoid).  hidden size 4. seems no beter.  0.314/0.316
  * Back to 2 layers all selu. seems worse.  0.320/0.322
  * Back to selu/linear.  
* Other things to try
  * More layers
  * wait longer
  * Different activations
  * Different neuron counts
  * In GNN, self loops?
  * Try with original imputer also

In [ ]:
import stellargraph as sg
sg.utils.plot_history(history)